#### [Weights and Biases](https://www.wandb.com/) is a tool for visualizing and tracking machine learning experiments.

In [1]:
# Import abilities
from ai import Chat, Vision, Audio, Images, Embeddings, FineTuner
import powers
import wandb
import time
from wandb.sdk.data_types.trace_tree import Trace
from datetime import datetime

# Adjust model globally here or modify in function calls. Vision model cannot be changed, for now ;) 
model = "gpt-3.5-turbo-1106"

In [2]:
def wandb_chain(chain_name, parent_span=None, root_span=None):

    end_time_ms = round(time.time() * 1000) # end time is now

    # Ensure contiguous spans.
    if parent_span:
        start_time_ms = parent_span.end_time_ms # start time is end time of parent span
    elif root_span and not parent_span:
        start_time_ms = root_span.end_time_ms   # start time is end time of root span
    else:
        start_time_ms = end_time_ms             # start time is now

    chain_span = Trace(
        name=chain_name,
        kind="chain",
        start_time_ms=start_time_ms,
        end_time_ms=end_time_ms,
    )

    if parent_span:
        parent_span.add_child(chain_span)       # add chain span as child of parent span.
        parent_span.end_time_ms = end_time_ms   # end time of parent span updated to end time of chain span.
    if not parent_span and root_span:
        root_span.add_child(chain_span)         # add chain span as direct child of root span.
        root_span.end_time_ms = end_time_ms     # end time of root span updated to end time of chain span.
    if not parent_span and not root_span: 
        root_span = chain_span                  # if no parent or root span, chain span is root span.

    return chain_span, parent_span, root_span

def wandb_llm(llm_name, inputs, outputs, parent_span=None, root_span=None, status="success", metadata=None):

    if root_span is None:
        Exception("Root span must be specified for LLM spans")

    end_time_ms = round(time.time() * 1000)

    # Ensure contiguous spans.
    if parent_span:
        start_time_ms = parent_span.end_time_ms  # start time is end time of parent span.
    elif root_span and not parent_span:
        start_time_ms = root_span.end_time_ms    # start time is end time of root span.
        
    llm_span = Trace(
        name=llm_name,
        kind="llm",
        status_code=status,
        start_time_ms=start_time_ms,
        end_time_ms=end_time_ms,
        inputs=inputs,
        outputs=outputs,
        metadata=metadata or {}
    )

    if parent_span:
        parent_span.add_child(llm_span)         # add llm span as child of parent span.
        parent_span.end_time_ms = end_time_ms   # end time of parent span updated to end time of llm span.
    if not parent_span and root_span:
        root_span.add_child(llm_span)           # add llm span as direct child of root span.

    root_span.end_time_ms = end_time_ms         # the root_span's end time is updated.

    return llm_span, parent_span, root_span

def wandb_tool(tool_name, inputs, outputs, parent_span=None, root_span=None, status="success", metadata=None):

    if root_span is None:
        Exception("Root span must be specified for tool spans")

    end_time_ms = round(time.time() * 1000)

    # Ensure contiguous spans.
    if parent_span:
        start_time_ms = parent_span.end_time_ms  # start time is end time of parent span.
    elif root_span and not parent_span: 
        start_time_ms = root_span.end_time_ms    # start time is end time of root span.

    tool_span = Trace(
        name=tool_name,
        kind="tool",
        status_code=status,
        start_time_ms=start_time_ms,
        end_time_ms=end_time_ms,
        inputs=inputs,
        outputs=outputs,
        metadata=metadata or {}
    )

    if parent_span:
        parent_span.add_child(tool_span)       # add tool span as child of parent span.
        parent_span.end_time_ms = end_time_ms  # end time of parent span updated to end time of tool span.
    if not parent_span and root_span:
        root_span.add_child(tool_span)         # add tool span as direct child of root span.

    root_span.end_time_ms = end_time_ms        # the root_span's end time is updated.
    return tool_span, parent_span, root_span


### Weights and Biases x OpenAI-Launchpad

Using wandb prompts and traces to visualize the flow in AI systems.

In [3]:
audio = Audio()
chat = Chat(model=model, system="Helpful assistant.")

wandb.init(project="openai-launchpad") ## Initialize wandb project

chain_name = "conversation"

chain_span, parent_span, root_span = wandb_chain(chain_name=chain_name, parent_span=None, root_span=None)

for i in range(5):

    # transcript = audio.record_and_transcribe()
    transcript = "This is a test."
    time.sleep(1)

    transcript_span, chain_span, root_span = wandb_tool(
        tool_name="transcription",
        inputs={"audio": "audio"},
        outputs={"transcript": transcript},
        parent_span=chain_span,
        root_span=root_span,
    )

    transcript_span.log(name=chain_name)


    if "quit" in transcript:
        break

    completion, messages = chat.chat_completion(transcript, speak=False, stream=True, memories=True)

    # Log the LLM span using the log_llm_span function.
    llm_span, _,  chain_span = wandb_llm(
        llm_name="chat",
        inputs={"transcript": transcript},
        outputs={"completion": completion, "messages": messages},
        parent_span=None,
        root_span=chain_span,
    )

    voice = "echo"
    # audio.speak(completion, voice=voice)
    time.sleep(2)

    # Log the speech tool span using the log_tool_span function.
    speech_span, _ , chain_span = wandb_tool(
        tool_name="speech",
        inputs={"completion": completion},
        outputs={"tool": "Completion spoken."},
        parent_span=llm_span,
        root_span=chain_span,
        metadata={"voice": voice}
    )

    speech_span.log(name=chain_name)

    llm_span.log(name=chain_name)

conversation_end_time_ms = round(datetime.now().timestamp() * 1000)
chain_span.end_time_ms = conversation_end_time_ms
chain_span.log(name=chain_name)

# End wandb run
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Initalized Audio class.
---------------------------------------------

Initalized Chat class with model gpt-3.5-turbo-1106
---------------------------------------------



wandb: Currently logged in as: samuel-shapley. Use `wandb login --relogin` to force relogin


Making Chat Completion API call...
---------------------------------------------

It looks like everything is working perfectly, how can I help you with today?

Making Chat Completion API call...
---------------------------------------------

It seems like everything is working fine. How may I assist you today?

Making Chat Completion API call...
---------------------------------------------

It looks like everything is working correctly. How can I assist you today?

Making Chat Completion API call...
---------------------------------------------

It seems like everything is working properly. How can I assist you today?

Making Chat Completion API call...
---------------------------------------------

It looks like everything is functioning as expected. How can I assist you further?



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
